In [ ]:
import numpy as np
import pandas as pd

from M4_experiment import M4_parser
from ESRNN import ESRNN

In [ ]:
import matplotlib.pyplot as plt
plt.style.use('ggplot')


def plot_prediction(y, y_hat, forecast=True):
    """
    """
    n_y = len(y)
    n_yhat = len(y_hat)
    ds_y = np.array(range(n_y))
    ds_yhat = np.array(range(n_y, n_y+n_yhat))

    plt.plot(ds_y, y, label = 'y')
    plt.plot(ds_yhat, y_hat, label='y_hat')
    plt.legend(loc='upper left')
    plt.show()

In [ ]:
X, y = M4_parser(dataset_name='Quarterly', mode='train', num_obs=1000)
print(X.head())
print(y[0:6])

In [ ]:
esrnn = ESRNN(max_epochs=0)
esrnn.fit(X,y)
pred_ini = esrnn.predict()

In [ ]:
unique_idx = 2
unique_id=X['unique_id'].unique()[unique_idx]

In [ ]:
plot_prediction(X[X['unique_id']==unique_id]['y'], pred_ini[pred_ini['unique_id']==unique_id]['y_hat'])

In [ ]:
esrnn = ESRNN(max_epochs=5)
esrnn.fit(X,y)
preds = esrnn.predict()

In [ ]:
plot_prediction(X[X['unique_id']==unique_id]['y'], preds[preds['unique_id']==unique_id]['y_hat'])